In [2]:
import cvxpy as cp
import numpy as np
import random

In [3]:
def queueGenerator(numQueue):
    listDataEquip, listDataPcard, listDataPtime, listDataPriority = [], [], [], []
    listEquip = ['4082', 'V93K']
    listPcard = ['AV44', 'FF44']

    for _ in range(numQueue):
        equip = random.choices(listEquip, weights=[0.8, 0.2])
        pcard = random.choices(listPcard, weights=[0.8, 0.2])
        # 0~100까지
        ptime = np.random.beta(1, 5, 1).item()*100
        priority = random.choices(range(0, 7), weights=[0.8, 0.01, 0.01, 0.01, 0.02, 0.05, 0.1])
        listDataEquip.append(equip)
        listDataPcard.append(pcard)
        listDataPtime.append(ptime)
        listDataPriority.append(priority)
    return listDataEquip, listDataPcard, listDataPtime, listDataPriority

numQueue = int(np.random.beta(1, 5, 1).item()*50 + 30)
listDataEquip, matrixPcard, p, priority = queueGenerator(numQueue)

In [4]:
len(listDataEquip)

32

In [ ]:
J = len(listDataEquip) # number of jobs
M = 14 # number of machines
r = 0.16 # release time
BIG_M = 1000
matrixEquip = np.array([[listDataEquip[j] == listDataEquip[m] for j in range(J)] for m in range(M)])
# s = cp.Variable(J, nonneg=True) # 시작 시간
# p = cp.Constant(listP)
C = cp.Variable(J, nonneg=True) # 완료 시간
z = cp.Variable((J, M), boolean=True) # 할당 여부
o = cp.Variable((J, J), boolean=True) # 순서 결정
y = cp.Variable((J, J), boolean=True) # 동일 장비


constraints = []

# 0. 초기 조건 (이미 할당된 Plan 들을 Job의 M번째까지 초기 조건으로 할당)
for j in range(J):
    for m in range(M):
        if not matrixEquip[m][j]:
            constraints.append(z[j, m] == 0)     
        if j == m:
            constraints.append(z[j, m] == 1)
            # for k in range(M, J):
            #     constraints.append(o[j, k] == 1)
            #     constraints.append(C[j] <= C[k] + BIG_M * (1 - z[k, m]))

# 2. Priority 조건
constraints += [o[l, j] == 0 for l in range(J) for j in range(J) if ((l >= M) and (j >= M) and (priority[l] < priority[j]))]

constraints += [o[l,j] + o[j,l] + y[l,j] == 1 for l in range(J) for j in range(J) if l < j]
constraints += [o[l,j] + o[j,k] + o[k,l] <= 2 for l in range(J) for j in range(J) for k in range(J) if l < j < k]
constraints += [z[l,i] + z[j,i] + y[l,j] <= 2 for i in range(M) for l in range(J) for j in range(J) if l < j]
constraints += [sum(z[l,i] for i in range(M)) == 1 for l in range(J)]
# constraints += [C[j] >= p[j]*z[j,i] for i in range(M) for j in range(J)]
constraints += [C[j] >= p[j] for j in range(J)]
constraints += [C[j] >= C[l] + (p[j] + r*int(matrixPcard[j] != matrixPcard[l]))*(o[l,j] + z[l,i] + z[j,i] - 2) - BIG_M*(1-o[l,j]) for i in range(M) for l in range(J) for j in range(J)]


obj = cp.Minimize(cp.mean(C))
# raise ValueError
prob = cp.Problem(obj, constraints)
# prob.solve(solver='CBC')
prob.solve(solver=cp.HIGHS)
print('최적해 상태: ', prob.status)
print('최적값: ', prob.value)


In [21]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

ganttData = []
for j in range(J):
    idMachine = int(np.argmax(z.value[j], axis=0))
    if (j >= 14) and (matrixPcard[j-1] != matrixPcard[j]):
        ganttData.append({
        'Job': f'Setup {j}',
        'Start': float(C.value[j] - p[j] - r*int(matrixPcard[j-1] != matrixPcard[j])),
        'End': float(C.value[j] - p[j]),
        'Machine': f'Machine {idMachine}'
    })
    ganttData.append({
        'Job': f'Job {j}',
        'Start': float(C.value[j] - p[j]),
        'End': float(C.value[j]),
        'Machine': f'Machine {idMachine}'
    })
dfGantt = pd.DataFrame(ganttData)

machine_list = sorted(dfGantt["Machine"].unique())
machine_to_y = {machine: idx for idx, machine in enumerate(machine_list)}

# Plotly 그래프 객체 초기화
fig = go.Figure()

# 각 작업/Setup을 개별 막대로 그림
for _, row in dfGantt.iterrows():
    fig.add_trace(go.Bar(
        x=[row["End"] - row["Start"]],
        y=[machine_to_y[row["Machine"]]],
        base=[row["Start"]],
        orientation='h',
        name=row["Job"],
        hovertext=f'{row["Job"]}: {row["Start"]} ~ {row["End"]}',
        hoverinfo='text',
        marker_color="lightcoral" if "Setup" in row["Job"] else "skyblue",
        showlegend=False
    ))

# y축 설정
fig.update_yaxes(
    tickvals=list(machine_to_y.values()),
    ticktext=list(machine_to_y.keys()),
    title="Machine",
    autorange="reversed"
)

# x축 수치형으로 설정
fig.update_xaxes(title="Time (unit)", type='linear')

# 기타 레이아웃
fig.update_layout(
    title="Gantt Chart (with numeric x-axis using Plotly)",
    barmode='stack',
    height=300 + 50 * len(machine_to_y)
)

fig.show()